In [15]:
!pip install kagglehub
!pip install numpy

You should consider upgrading via the 'D:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\project\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\project\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import shutil
import os
import kagglehub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm

In [8]:
# Download latest version
path = kagglehub.dataset_download("deepshah16/song-lyrics-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\jaeseokk\.cache\kagglehub\datasets\deepshah16\song-lyrics-dataset\versions\5


In [9]:
current_directory = os.getcwd()
print("Current Path:", current_directory)

destination_path = os.path.join(current_directory, "song-lyrics-dataset")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Dataset directory has been copied to:", destination_path)

Current Path: d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project
Dataset directory has been copied to: d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset


Lyrics datasets have been saved in `song-lyrics-dataset/csv`.

In [24]:
csv_path = os.path.join(current_directory, "song-lyrics-dataset", "csv")

artists = os.listdir(csv_path)
for artist in artists:
    lyrics_data = os.path.join(csv_path,artist)
    print(lyrics_data)
    df1 = pd.read_csv(lyrics_data)

# 다 합치는 코드 작성해야함
    
df1.head()


d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ArianaGrande.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Beyonce.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BillieEilish.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BTS.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CardiB.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CharliePuth.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ColdPlay.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Drake.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\DuaLipa.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\EdSheeran.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Eminem.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Pro

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...
